# Objectif

L'objectif principal de ce fichier est de réorganiser le dossier "**Features_MFCC**", qui contient chaque langue avec des dossiers (train/test) contenant les caractéristiques MFCC extraites par **la partie I (reconnaissance de la langue)**. Ces données sont initialement structurées par langue et type (train/test). Nous souhaitons les réorganiser en une structure où les données sont organisées par langue, genre du locuteur, identifiant du locuteur, puis type (train/test) pour chaque locuteur. Elles seront ensuite stockées dans le dossier "**Features_MFCC_Locuteur**" pour **la partie II (reconnaissance du locuteur)**.

Importation des bibliothèques

In [ ]:
import os
import numpy as np
import pandas as pd
import re # Pour découper les noms de fichiers audio
import shutil # Pour déplacer des fichiers

from google.colab import drive
drive.mount('/content/drive') # Monter (Mount) Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Le répertoire de données qui contient les caractéristiques MFCC
- Chargement des données MFCC sauvegardées

In [ ]:
# Chemin vers le dossier MFCC dans Google Drive
mfcc_dir = '/content/drive/My Drive/Reconnaissance/Features_MFCC'

langues = os.listdir(mfcc_dir)
print(langues)

['French', 'arabic', 'Japanese', 'Spanish', 'English']


- **Extraction d'informations** : Cette fonction extrait le nom du locuteur et son genre (homme/femme) à partir des noms de fichiers audio
- Si le nom du fichier est **Ar_H1_1.wav**
- Le fichier est découpé en segments : ["Ar", "H1", "1.wav"]

In [ ]:
def extract_speaker_identifier(segment_label):

    speaker_id = None
    # Découper le nom de fichier en segments.
    segments = re.split(r'[_ ]', segment_label)
    segments[1] = segments[1].upper()  # Convertir l'identifiant du locuteur en majuscules.

    # Gestion des fichiers ayant 3 parties après découpage.
    if len(segments) == 3:
        # Identifier les locuteurs avec un identifiant de plus d'un caractère.
        if len(segments[1]) > 1:
            speaker_id = segments[1]

    # Gestion des fichiers ayant 4 parties après découpage.
    elif len(segments) == 4:
        # Traiter les locuteurs avec un identifiant court (1 caractère).
        if len(segments[1]) == 1:
            speaker_id = segments[1] + segments[2]  # Construire l'identifiant complet du locuteur.
        else:
            speaker_id = segments[1]

    return speaker_id


Structurer les fichiers audio par langue, genre (homme ou femme), et locuteur, en distinguant les fichiers d'entraînement et de test.

Cette fonction organise les données du répertoire en un dictionnaire structuré par :
- **Langues** : Chaque langue est une clé du dictionnaire.
- **Genres (homme/femme)** : Les locuteurs sont classés par genre.
- **Locuteurs** : Pour chaque locuteur, les fichiers sont séparés en "train" (entraînement) et "test".

In [ ]:
def organize_speaker_data(data_dir):

    # Dictionnaire pour stocker les données organisées par langue, genre, et locuteur
    organized_data = {}

    # Liste des langues dans le dossier de données
    language_list = os.listdir(data_dir)

    # Parcours de chaque langue pour organiser les données
    for language in language_list:

        # Initialisation de la structure pour une langue : contient 'male' et 'female'
        organized_data[language] = {
            'Homme': {},
            'Femme': {}
        }

        # Parcours des fichiers d'entraînement
        train_folder = os.path.join(data_dir, language, 'train')
        train_segments = os.listdir(train_folder)

        for segment in train_segments:
            # Extraction du locuteur à partir du nom du fichier
            speaker = extract_speaker_identifier(segment)

            # Vérification si le locuteur est un homme
            if "H" in speaker:
                # Si le locuteur n'existe pas encore dans la structure, on l'ajoute
                if organized_data[language]['Homme'].get(speaker) is None:
                    organized_data[language]['Homme'][speaker] = {
                            'train': [segment]  # Initialisation avec le fichier actuel
                        }
                else:
                    # Sinon, on ajoute le fichier actuel à la liste d'entraînement existante
                    organized_data[language]['Homme'][speaker]['train'].append(segment)

            else:
                # Même traitement pour les locuteurs de genre femme
                if organized_data[language]['Femme'].get(speaker) is None:
                    organized_data[language]['Femme'][speaker] = {
                            'train': [segment]
                        }
                else:
                    organized_data[language]['Femme'][speaker]['train'].append(segment)

        # Parcours des fichiers de test
        test_folder = os.path.join(data_dir, language, 'test')
        test_segments = os.listdir(test_folder)

        for segment in test_segments:
            # Extraction du locuteur à partir du nom du fichier
            speaker = extract_speaker_identifier(segment)

            # Vérification si le locuteur est un homme
            if "H" in speaker:
                # Si la clé 'test' n'existe pas encore, on l'ajoute
                if organized_data[language]['Homme'][speaker].get('test') is None:
                    organized_data[language]['Homme'][speaker]['test'] = [segment]
                else:
                    # Sinon, on ajoute le fichier actuel à la liste de test existante
                    organized_data[language]['Homme'][speaker]['test'].append(segment)

            else:
                # Même traitement pour les locuteurs de genre femme
                if organized_data[language]['Femme'][speaker].get('test') is None:
                    organized_data[language]['Femme'][speaker]['test'] = [segment]
                else:
                    organized_data[language]['Femme'][speaker]['test'].append(segment)

    # Renvoi des données organisées par langue, genre, et locuteur
    return organized_data


- Cette structure de données stocke des informations sur les fichiers audio organisés par langue, genre du locuteur, identifiant du locuteur et type de données (train/test)

In [ ]:
# Les données organisées par langue, genre, et locuteur
organized_data = organize_speaker_data(mfcc_dir)

organized_data['French']['Homme']['H1']['train']
organized_data['French']['Homme']['H1']
organized_data['French']['Homme']
organized_data['French']


{'Homme': {'H1': {'train': ['Fr_H1_2_10mn.mfcc'],
   'test': ['Fr_H1_1_5mn30s.mfcc']},
  'H2': {'train': ['Fr_H2_2_10mn.mfcc'], 'test': ['Fr_H2_1_6mn.mfcc']},
  'H3': {'train': ['Fr_H3_2_11mn.mfcc'], 'test': ['Fr_H3_1_5mn50s.mfcc']}},
 'Femme': {'F1': {'train': ['Fr_F1_2_5m50s.mfcc'],
   'test': ['Fr_F1_1_5m11s.mfcc']},
  'F4': {'train': ['Fr_F4_4_1m18s.mfcc',
    'Fr_F4_1_1m34s.mfcc',
    'Fr_F4_3_1m20s.mfcc',
    'Fr_F4_5_1m39s.mfcc'],
   'test': ['Fr_F4_2_1m.mfcc', 'Fr_F4_6_1m14.mfcc']},
  'F3': {'train': ['Fr_F3_4_4m45s.mfcc',
    'Fr_F3_1_3m27s.mfcc',
    'Fr_F3_3_3m20s.mfcc'],
   'test': ['Fr_F3_2_1m8s.mfcc']},
  'F5': {'train': ['Fr_F5_1_5m19s.mfcc'], 'test': ['Fr_F5_2_3m19s.mfcc']},
  'F6': {'train': ['Fr_F6_2_5m20s.mfcc'], 'test': ['Fr_F6_1_3m23s.mfcc']}}}

- Parcours des données structurées, organisées par langue, genre et locuteur
- Pour chaque langue, un répertoire est créé, puis pour chaque genre (homme/femme), un sous-répertoire est créé.
- Ensuite, pour chaque locuteur, un répertoire est créé contenant les sous-répertoires 'train' et 'test', où les segments de données sont copiés depuis le répertoire d'origine vers le répertoire spécifique du locuteur.

In [ ]:
### N'exécutez pas cette cellule, car elle a déjà été exécutée et les résultats sont déjà enregistrés et stockés dans Google Drive. Cela prend beaucoup de temps.

# Chemin vers le dossier dans Google Drive
base_dir = "/content/drive/My Drive/Reconnaissance/Features_MFCC_Locuteur"
os.makedirs(base_dir, exist_ok=True)

for language, categories in organized_data.items():
    # Création du répertoire de la langue dans Google Drive
    language_dir = os.path.join(base_dir, language)
    os.makedirs(language_dir, exist_ok=True)

    # Parcours des genres (homme/femme)
    for category, speakers in categories.items():
        # Création du répertoire pour chaque genre (homme/femme)
        category_dir = os.path.join(language_dir, category)
        os.makedirs(category_dir, exist_ok=True)

        # Parcours des locuteurs
        for speaker, data in speakers.items():
            # Création du répertoire pour chaque locuteur
            speaker_dir = os.path.join(category_dir, speaker)
            os.makedirs(speaker_dir, exist_ok=True)

            # Création du sous-répertoire 'train' pour les fichiers d'entraînement
            speaker_train_dir = os.path.join(speaker_dir, 'train')
            os.makedirs(speaker_train_dir, exist_ok=True)

            # Spécification du répertoire source pour les fichiers d'entraînement dans Google Drive
            language_train_dir = os.path.join("/content/drive/My Drive/Reconnaissance/Features_MFCC", language, 'train')

            # Copie des segments d'entraînement dans le répertoire du locuteur sur Google Drive
            for segment in data['train']:
                source = os.path.join(language_train_dir, segment)
                destination = os.path.join(speaker_train_dir, segment)
                shutil.copy(source, destination)

            # Création du sous-répertoire 'test' pour les fichiers de test
            speaker_test_dir = os.path.join(speaker_dir, 'test')
            os.makedirs(speaker_test_dir, exist_ok=True)

            # Spécification du répertoire source pour les fichiers de test dans Google Drive
            language_test_dir = os.path.join("/content/drive/My Drive/Reconnaissance/Features_MFCC", language, 'test')

            # Copie des segments de test dans le répertoire du locuteur sur Google Drive
            for segment in data['test']:
                source = os.path.join(language_test_dir, segment)
                destination = os.path.join(speaker_test_dir, segment)
                shutil.copy(source, destination)


- Affiche la structure du répertoire de manière récursive

In [ ]:
def print_directory_structure(root_dir, indent=""):
    for item in os.listdir(root_dir):
        item_path = os.path.join(root_dir, item)
        if os.path.isdir(item_path):  # Vérifier si c'est un dossier
            print(f"{indent}- {item}")
            print_directory_structure(item_path, indent + "  ")  # Appel récursif pour les sous-dossiers

# Le chemin d'accès
root_dir = "/content/drive/My Drive/Reconnaissance/Features_MFCC_Locuteur"
print_directory_structure(root_dir)

- English
  - Femme
    - F12
      - train
      - test
    - F11
      - train
      - test
    - F13
      - train
      - test
    - F14
      - train
      - test
    - F15
      - train
      - test
    - F16
      - train
      - test
    - F17
      - train
      - test
    - F18
      - train
      - test
    - F19
      - train
      - test
    - F2
      - train
      - test
    - F20
      - train
      - test
    - F21
      - train
      - test
    - F3
      - train
      - test
    - F4
      - train
      - test
    - F5
      - train
      - test
    - F7
      - train
      - test
    - F6
      - train
      - test
    - F8
      - train
      - test
    - F9
      - train
      - test
    - F1
      - train
      - test
    - F10
      - train
      - test
  - Homme
    - H13
      - train
      - test
    - H14
      - train
      - test
    - H12
      - train
      - test
    - H15
      - train
      - test
    - H16
      - train
      - test
    - H17
      -

- Compte le nombre de fichiers .mfcc d'entraînement et de test pour chaque locuteur, en tenant compte du genre et de la langue, après la réorganisation des caractéristiques MFCC.

In [ ]:
def count_mfcc_files(root_dir):

  file_counts = {}

  for language in os.listdir(root_dir):
    file_counts[language] = {}
    for gender in os.listdir(os.path.join(root_dir, language)):
      file_counts[language][gender] = {}
      for speaker in os.listdir(os.path.join(root_dir, language, gender)):
        file_counts[language][gender][speaker] = {}
        for data_type in ["train", "test"]:
          speaker_dir = os.path.join(root_dir, language, gender, speaker, data_type)
          file_counts[language][gender][speaker][data_type] = len([
              f for f in os.listdir(speaker_dir) if f.endswith(".mfcc")
          ])

  return file_counts

# Utilisation de la fonction
root_dir = "/content/drive/My Drive/Reconnaissance/Features_MFCC_Locuteur"
file_counts = count_mfcc_files(root_dir)

# Affichage des résultats
for language, genders in file_counts.items():
  print(f"Langue: {language}")
  for gender, speakers in genders.items():
    print(f"  Genre: {gender}")
    for speaker, counts in speakers.items():
      print(f"    Locuteur: {speaker}")
      print(f"      Train: {counts['train']}")
      print(f"      Test: {counts['test']}")

Langue: English
  Genre: Femme
    Locuteur: F12
      Train: 43
      Test: 21
    Locuteur: F11
      Train: 27
      Test: 14
    Locuteur: F13
      Train: 60
      Test: 30
    Locuteur: F14
      Train: 39
      Test: 19
    Locuteur: F15
      Train: 63
      Test: 32
    Locuteur: F16
      Train: 51
      Test: 26
    Locuteur: F17
      Train: 39
      Test: 19
    Locuteur: F18
      Train: 50
      Test: 25
    Locuteur: F19
      Train: 43
      Test: 21
    Locuteur: F2
      Train: 37
      Test: 18
    Locuteur: F20
      Train: 28
      Test: 14
    Locuteur: F21
      Train: 63
      Test: 31
    Locuteur: F3
      Train: 52
      Test: 26
    Locuteur: F4
      Train: 48
      Test: 24
    Locuteur: F5
      Train: 51
      Test: 26
    Locuteur: F7
      Train: 64
      Test: 32
    Locuteur: F6
      Train: 37
      Test: 18
    Locuteur: F8
      Train: 53
      Test: 27
    Locuteur: F9
      Train: 39
      Test: 20
    Locuteur: F1
      Train: 43
      Test: 2